# 文字识别

百度百科:利用计算机自动识别字符的技术，是模式识别应用的一个重要领域。人们在生产和生活中，要处理大量的文字、报表和文本。为了减轻人们的劳动，提高处理效率，50年代开始探讨一般文字识别方法，并研制出光学字符识别器。60年代出现了采用磁性墨水和特殊字体的实用机器。60年代后期，出现了多种字体和手写体文字识别机，其识别精度和机器性能都基本上能满足要求。如用于信函分拣的手写体数字识别机和印刷体英文数字识别机。70年代主要研究文字识别的基本理论和研制高性能的文字识别机，并着重于汉字识别的研究。

# 开始我们的实验
(1)登录百度AI开发平台，选择“文字识别”，使用“创建应用”建立一个文字识别应用。
获取应用的AppID、API Key、Secret Key，注意:Secret Key要先选择显示，然后复制其现实的字符。

例如我创建的人脸识别应用获取的AppID、API Key、Secret Key如下：

    """ 公开课版本的限定量文字识别，需要更改为自己的api接口 """
    word_APP_ID = '21161335'
    word_API_KEY = 'h45HUETBcSNTHrDldK7wjGL7'
    word_SECRET_KEY = 'GtnhMLMQ8e670ptptAvPzhbCBikwuoOL'

百度AI开发平台注册链接:https://login.bce.baidu.com/?account=&redirect=http%3A%2F%2Fconsole.bce.baidu.com%2F%3Ffromai%3D1#/aip/overview

(2)熟悉我封装的百度AI文字识别的模块
在代码中输入如下代码，导入我的封装的模块

    import sys
    sys.path.append('../baidu_api_lib')
    from baidu_picture import baidu_picture_2_msg
    封装库提供两个方法：初始化方法，文字识别方法。

<1>初始化方法，需要传入三个参数，就是上步我们创建“文字识别”应用获取的三个参数
参数信息如下：

    """ 公开课版本的限定量文字识别，需要更改为自己的api接口 """
    word_APP_ID = '21161335'
    word_API_KEY = 'h45HUETBcSNTHrDldK7wjGL7'
    word_SECRET_KEY = 'GtnhMLMQ8e670ptptAvPzhbCBikwuoOL'
调用方式：

    # 传入百度AI的参数
    pic_msg = baidu_picture_2_msg(word_APP_ID, word_API_KEY, word_SECRET_KEY)
    初始化完毕后，获取pic_msg对象，操作该对象就能进行文字识别。

<2>文字识别方法
这是我封装百度文字识别方法的函数原型：

    #从百度AI获取图片分析结果
    response = pic_msg.pic_2_msg(baidu_request_url, 'read_word.jpg') 

pic_2_msg包含2个参数：

baidu_request_url:百度AI的调用接口，
   
    #百度AI的调用url
    baidu_request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/general_basic"
    
    'read_word.jpg':进行人脸识别的图片。
    返回值:response为文字识别的信息

    response.json()["words_result"]:识别结果数组
    response.json()["words_result_num"]:识别结果数，表示words_result的元素个数
    response.json()["words_result"]["words"]:文字
    ...还有不少参数，通过print(response.json())可以发现

In [ ]:
######################################################
#
# 先感受下小派的过目不忘,选择本cell，按shirt+enter键运行本模块
#
######################################################
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright (c) 2020 Taste all Pi.
#
# Licensed under the GNU General Public License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.gnu.org/licenses/gpl-2.0.html
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#导入标准库
import sys
import os
from playsound import playsound
import cv2 as cv
import time
from multiprocessing import Process, Queue
import multiprocessing

#导入自定义库
sys.path.append('../baidu_api_lib')
from baidu_picture import baidu_picture_2_msg
from baidu_sound import baidu_word_2_sound

""" 公开课版本的限定量文字识别，需要更改为自己的api接口 """
word_APP_ID = '21161335'
word_API_KEY = 'h45HUETBcSNTHrDldK7wjGL7'
word_SECRET_KEY = 'GtnhMLMQ8e670ptptAvPzhbCBikwuoOL'

#百度AI的调用url
baidu_request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/general_basic"

#　获取摄像头图像
def camera_frame_func(task_name, mult_queue1, mydict):
    
    # 创建一个VideoCapture对象
    capture = cv.VideoCapture(0) 
    
    #更新人脸识别信息
    mydict["get_pic"] = "false"
    
    #　给出提示信息
    print(task_name + "任务启动")
    
    try:
        while True:
            # 一帧一帧读取视频
            ret, frame = capture.read()
            
            #将拍摄到的图片发送到消息队列中
            if mydict["get_pic"] == "true":
                mydict["get_pic"] = "false"
                mult_queue1.put(frame)

            # 本地显示视频图像
            cv.imshow('real_time picture', frame) 
            cv.waitKey(1)

    except KeyboardInterrupt:
        # 释放cap,销毁窗口
        capture.release()    
        #关闭显示窗口
        cv.destroyAllWindows() 
        print(task_name + "任务被终止")
        
#处理图像
def proc_frame_func(task_name, mult_queue, mydict):
    #　给出提示信息
    print(task_name + "任务启动")
    
    # 传入百度AI的参数
    pic_msg = baidu_picture_2_msg(word_APP_ID, word_API_KEY, word_SECRET_KEY)
    
    #传入百度AI的参数
    word_2_sound = baidu_word_2_sound(word_APP_ID, word_API_KEY, word_SECRET_KEY)
    word_2_sound.trans_word_to_sound("咱们比拼下谁记得多，让我们开始吧",'tst_sound.mp3')
    os.system('mplayer ' + 'tst_sound.mp3')
    time.sleep(2)
    
    try:
       
        while True:
            #给出语音提示
            word_2_sound.trans_word_to_sound("我正在速记文字",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(2)
            word_2_sound.trans_word_to_sound("3",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)
            
            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #给出语音提示
            word_2_sound.trans_word_to_sound("2",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)

            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #给出语音提示
            word_2_sound.trans_word_to_sound("1",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)
            
            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #关闭显示窗口
            cv.destroyAllWindows()            
            
            #给出语音提示
            word_2_sound.trans_word_to_sound("你记下来了吗?反正我记下来了，可以把书拿走了，我来背诵,你来检查",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            
            # 写入图片
            cv.imwrite('read_word.jpg',frame)
            
            #从百度AI获取图片分析结果
            response = pic_msg.pic_2_msg(baidu_request_url, 'read_word.jpg')
            
            #给出百度AI分析的数据
            print(response.json())
            
            #获取分析结果中的有效文字列表
            word_list = response.json()['words_result']

            #将识别到的文字组合到一起，存储在all_word中
            all_word = ''
            for k in word_list:
                all_word += k['words']
                all_word += ','
                
            print(all_word)
            
            #背诵文章
            if all_word:
                #给出语音提示
                word_2_sound.trans_word_to_sound(all_word,'tst_sound.mp3')
                os.system('mplayer ' + 'tst_sound.mp3')
                
                #给出语音提示
                word_2_sound.trans_word_to_sound("我都背下来了吧，我都有点小骄傲",'tst_sound.mp3')
                os.system('mplayer ' + 'tst_sound.mp3')
                
            else:
                #给出语音提示
                word_2_sound.trans_word_to_sound("你在跟我开玩笑吗?不要晃动书，我会晕的",'tst_sound.mp3')
                os.system('mplayer ' + 'tst_sound.mp3')
                
            #给出语音提示
            word_2_sound.trans_word_to_sound("不服就翻下一页，我们继续比拼",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')    
            time.sleep(3)
            
    except KeyboardInterrupt:
        os.remove('tst_sound.mp3')
        os.remove('read_word.jpg')
        print(task_name + "任务被终止")
        
if __name__ == "__main__":
    try:
        
        mydict=multiprocessing.Manager().dict()
        
        #　定义传递图像队列和传递图像处理结果队列
        q_frame = Queue()
        
        #　采集摄像头进程、处理图片进程、播报语音信息
        get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))
        proc_frame       = Process(target=proc_frame_func, args=("处理图像", q_frame, mydict))
        
        # 启动任务
        get_camera_frame.start()
        proc_frame.start()

    except KeyboardInterrupt:
        print("任务被终止了")

# 让我们临摹代码，学习“文字识别”功能

In [ ]:
# #导入标准库
# import sys

# import os

# from playsound import playsound

# import cv2 as cv

# import time

# from multiprocessing import Process, Queue

# import multiprocessing


# #导入自定义库
# sys.path.append('../baidu_api_lib')

# from baidu_picture import baidu_picture_2_msg

# from baidu_sound import baidu_word_2_sound


# """ 公开课版本的限定量文字识别，需要更改为自己的api接口 """
# word_APP_ID = '21161335'

# word_API_KEY = 'h45HUETBcSNTHrDldK7wjGL7'

# word_SECRET_KEY = 'GtnhMLMQ8e670ptptAvPzhbCBikwuoOL'


# #百度AI的调用url
# baidu_request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/general_basic"


# #　获取摄像头图像
# def camera_frame_func(task_name, mult_queue1, mydict):

    
#     # 创建一个VideoCapture对象
#     capture = cv.VideoCapture(0) 

    
#     #更新人脸识别信息
#     mydict["get_pic"] = "false"

    
#     #　给出提示信息
#     print(task_name + "任务启动")

    
#     try:

#         while True:

#             # 一帧一帧读取视频
#             ret, frame = capture.read()

            
#             #将拍摄到的图片发送到消息队列中
#             if mydict["get_pic"] == "true":

#                 mydict["get_pic"] = "false"

#                 mult_queue1.put(frame)


#             # 本地显示视频图像
#             cv.imshow('real_time picture', frame) 

#             cv.waitKey(1)


#     except KeyboardInterrupt:

#         # 释放cap,销毁窗口
#         capture.release()   

#         #关闭显示窗口
#         cv.destroyAllWindows() 

#         print(task_name + "任务被终止")

        
# #处理图像
# def proc_frame_func(task_name, mult_queue, mydict):

#     #　给出提示信息
#     print(task_name + "任务启动")

    
#     # 传入百度AI的参数
#     pic_msg = baidu_picture_2_msg(word_APP_ID, word_API_KEY, word_SECRET_KEY)

    
#     #传入百度AI的参数
#     word_2_sound = baidu_word_2_sound(word_APP_ID, word_API_KEY, word_SECRET_KEY)

#     word_2_sound.trans_word_to_sound("咱们比拼下谁记得多，让我们开始吧",'tst_sound.mp3')

#     os.system('mplayer ' + 'tst_sound.mp3')

#     time.sleep(2)

    
#     try:
       
#         while True:

#             #给出语音提示
#             word_2_sound.trans_word_to_sound("我正在速记文字",'tst_sound.mp3')

#             os.system('mplayer ' + 'tst_sound.mp3')

#             time.sleep(2)

#             word_2_sound.trans_word_to_sound("3",'tst_sound.mp3')

#             os.system('mplayer ' + 'tst_sound.mp3')

#             time.sleep(1)

            
#             #从队列中获取图片，显示图像质量
#             mydict["get_pic"] = "true"

#             frame = mult_queue.get()

#             cv.waitKey(1)

            
#             #给出语音提示
#             word_2_sound.trans_word_to_sound("2",'tst_sound.mp3')

#             os.system('mplayer ' + 'tst_sound.mp3')

#             time.sleep(1)


#             #从队列中获取图片，显示图像质量
#             mydict["get_pic"] = "true"

#             frame = mult_queue.get()

#             cv.waitKey(1)

            
#             #给出语音提示
#             word_2_sound.trans_word_to_sound("1",'tst_sound.mp3')

#             os.system('mplayer ' + 'tst_sound.mp3')

#             time.sleep(1)

            
#             #从队列中获取图片，显示图像质量
#             mydict["get_pic"] = "true"

#             frame = mult_queue.get()

#             cv.waitKey(1)

            
#             #关闭显示窗口
#             cv.destroyAllWindows()    

            
#             #给出语音提示
#             word_2_sound.trans_word_to_sound("你记下来了吗?反正我记下来了，可以把书拿走了，我来背诵,你来检查",'tst_sound.mp3')

#             os.system('mplayer ' + 'tst_sound.mp3')

            
#             # 写入图片
#             cv.imwrite('read_word.jpg',frame)

            
#             #从百度AI获取图片分析结果
#             response = pic_msg.pic_2_msg(baidu_request_url, 'read_word.jpg')

            
#             #给出百度AI分析的数据
#             print(response.json())

            
#             #获取分析结果中的有效文字列表
#             word_list = response.json()['words_result']


#             #将识别到的文字组合到一起，存储在all_word中
#             all_word = ''

#             for k in word_list:

#                 all_word += k['words']

#                 all_word += ','

                
#             print(all_word)

            
#             #背诵文章
#             if all_word:

#                 #给出语音提示
#                 word_2_sound.trans_word_to_sound(all_word,'tst_sound.mp3')

#                 os.system('mplayer ' + 'tst_sound.mp3')

                
#                 #给出语音提示
#                 word_2_sound.trans_word_to_sound("我都背下来了吧，我都有点小骄傲",'tst_sound.mp3')

#                 os.system('mplayer ' + 'tst_sound.mp3')

                
#             else:

#                 #给出语音提示
#                 word_2_sound.trans_word_to_sound("你在跟我开玩笑吗?不要晃动书，我会晕的",'tst_sound.mp3')

#                 os.system('mplayer ' + 'tst_sound.mp3')

                
#             #给出语音提示
#             word_2_sound.trans_word_to_sound("不服就翻下一页，我们继续比拼",'tst_sound.mp3')

#             os.system('mplayer ' + 'tst_sound.mp3')    

#             time.sleep(3)

            
#     except KeyboardInterrupt:

#         os.remove('tst_sound.mp3')

#         os.remove('read_word.jpg')

#         print(task_name + "任务被终止")

        
# if __name__ == "__main__":

#     try:

        
#         mydict=multiprocessing.Manager().dict()

        
#         #　定义传递图像队列和传递图像处理结果队列
#         q_frame = Queue()

        
#         #　采集摄像头进程、处理图片进程、播报语音信息
#         get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))

#         proc_frame       = Process(target=proc_frame_func, args=("处理图像", q_frame, mydict))

        
#         # 启动任务
#         get_camera_frame.start()

#         proc_frame.start()


#     except KeyboardInterrupt:
#         print("任务被终止了")


# 课后练习
１、记忆书上的课文对小派还是很容易的，让我使用尝试下我们手写的文章，也让小派过目不忘的背诵下

In [ ]:
#导入标准库
import sys
import os
from playsound import playsound
import cv2 as cv
import time
from multiprocessing import Process, Queue
import multiprocessing

#导入自定义库
sys.path.append('../baidu_api_lib')
from baidu_picture import baidu_picture_2_msg
from baidu_sound import baidu_word_2_sound

""" 公开课版本的限定量文字识别，需要更改为自己的api接口 """
word_APP_ID = '21161335'
word_API_KEY = 'h45HUETBcSNTHrDldK7wjGL7'
word_SECRET_KEY = 'GtnhMLMQ8e670ptptAvPzhbCBikwuoOL'

#百度AI的调用url 手写识别的百度AI接口
baidu_request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/handwriting"

#　获取摄像头图像
def camera_frame_func(task_name, mult_queue1, mydict):
    
    # 创建一个VideoCapture对象
    capture = cv.VideoCapture(0) 
    
    #更新人脸识别信息
    mydict["get_pic"] = "false"
    
    #　给出提示信息
    print(task_name + "任务启动")
    
    try:
        while True:
            # 一帧一帧读取视频
            ret, frame = capture.read()
            
            #将拍摄到的图片发送到消息队列中
            if mydict["get_pic"] == "true":
                mydict["get_pic"] = "false"
                mult_queue1.put(frame)

            # 本地显示视频图像
            cv.imshow('real_time picture', frame) 
            cv.waitKey(1)

    except KeyboardInterrupt:
        # 释放cap,销毁窗口
        capture.release()    
        #关闭显示窗口
        cv.destroyAllWindows() 
        print(task_name + "任务被终止")
        
#处理图像
def proc_frame_func(task_name, mult_queue, mydict):
    #　给出提示信息
    print(task_name + "任务启动")
    
    # 传入百度AI的参数
    pic_msg = baidu_picture_2_msg(word_APP_ID, word_API_KEY, word_SECRET_KEY)
    
    #传入百度AI的参数
    word_2_sound = baidu_word_2_sound(word_APP_ID, word_API_KEY, word_SECRET_KEY)
    word_2_sound.trans_word_to_sound("咱们比拼下谁记得多，让我们开始吧",'tst_sound.mp3')
    os.system('mplayer ' + 'tst_sound.mp3')
    time.sleep(2)
    
    try:
       
        while True:
            #给出语音提示
            word_2_sound.trans_word_to_sound("我正在速记文字",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(2)
            word_2_sound.trans_word_to_sound("3",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)
            
            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #给出语音提示
            word_2_sound.trans_word_to_sound("2",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)

            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #给出语音提示
            word_2_sound.trans_word_to_sound("1",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)
            
            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #关闭显示窗口
            cv.destroyAllWindows()            
            
            #给出语音提示
            word_2_sound.trans_word_to_sound("你记下来了吗?反正我记下来了，可以把书拿走了，我来背诵,你来检查",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            
            # 写入图片
            cv.imwrite('read_word.jpg',frame)
            
            #从百度AI获取图片分析结果
            response = pic_msg.pic_2_msg(baidu_request_url, 'read_word.jpg')
            
            #给出百度AI分析的数据
            print(response.json())
            
            #获取分析结果中的有效文字列表
            word_list = response.json()['words_result']

            #将识别到的文字组合到一起，存储在all_word中
            all_word = ''
            for k in word_list:
                all_word += k['words']
                all_word += ','
                
            print(all_word)
            
            #背诵文章
            if all_word:
                #给出语音提示
                word_2_sound.trans_word_to_sound(all_word,'tst_sound.mp3')
                os.system('mplayer ' + 'tst_sound.mp3')
                
                #给出语音提示
                word_2_sound.trans_word_to_sound("我都背下来了吧，我都有点小骄傲",'tst_sound.mp3')
                os.system('mplayer ' + 'tst_sound.mp3')
                
            else:
                #给出语音提示
                word_2_sound.trans_word_to_sound("你在跟我开玩笑吗?不要晃动书，我会晕的",'tst_sound.mp3')
                os.system('mplayer ' + 'tst_sound.mp3')
                
            #给出语音提示
            word_2_sound.trans_word_to_sound("不服就翻下一页，我们继续比拼",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')    
            time.sleep(3)
            
    except KeyboardInterrupt:
        os.remove('tst_sound.mp3')
        os.remove('read_word.jpg')
        print(task_name + "任务被终止")
        
if __name__ == "__main__":
    try:
        
        mydict=multiprocessing.Manager().dict()
        
        #　定义传递图像队列和传递图像处理结果队列
        q_frame = Queue()
        
        #　采集摄像头进程、处理图片进程、播报语音信息
        get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))
        proc_frame       = Process(target=proc_frame_func, args=("处理图像", q_frame, mydict))
        
        # 启动任务
        get_camera_frame.start()
        proc_frame.start()

    except KeyboardInterrupt:
        print("任务被终止了")